# VR Assignment 3 - Part B

## Initial Imports

In [1]:
import os
import torch
import torch.nn
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F 
import torchvision.utils as utils
import cv2 
import matplotlib.pyplot as plt
import numpy as np 
from PIL import Image, ImageOps
import argparse
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

## Data Reads

In [2]:
TrainDirectory = "../input/spider-dataset/train"
TestDirectory = "../input/spider-dataset/test"
Categories = os.listdir(TrainDirectory)
TestCategories = os.listdir(TestDirectory)


# TrainDirectory = "../input/bikesvshorses/Assignment2_BikeHorses/"
# Categories = os.listdir(TrainDirectory)

## Initial Data Transform

In [3]:
data_transforms = transforms.Compose([
        transforms.Resize((224,224)),      
        transforms.ToTensor(),             
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  
    ])

## Creating a list of transformed train and test images

In [4]:
Data = []
labels = []

for category in Categories:
    fileLabel = Categories.index(category)
    for file in os.listdir(os.path.join(TrainDirectory, category)):
        img =  Image.open(os.path.join(TrainDirectory, category, file))
        labels.append(fileLabel)
        transformed_img = data_transforms(img)
        Data.append(torch.unsqueeze(transformed_img, 0))    

In [5]:
test = []
testLabels = []

for category in TestCategories:
    fileLabel = TestCategories.index(category)
    for file in os.listdir(os.path.join(TestDirectory, category)):
        img =  Image.open(os.path.join(TestDirectory, category, file))
        testLabels.append(fileLabel)
        transformed_img = data_transforms(img)
        test.append(torch.unsqueeze(transformed_img, 0))    


## Initialising pretrained alexnet

In [6]:
alexnet = models.alexnet(pretrained = True)
alexnet.eval()

## Feature extraction using pretrained Alexnet

In [7]:
features = []
i = 0
for img in Data:
    with torch.no_grad():
        feature = alexnet(img).detach().numpy()
    features.append(feature)
    if i%100 == 0:
        print(str(i) + " training images appended")
    i+=1
print("All Training Images Appended!")

In [8]:
i = 0
testFeatures = []
for img in test:
    with torch.no_grad():
        feature = alexnet(img).detach().numpy()
    testFeatures.append(feature)    
    if i%10 == 0:
        print(str(i) + " testing images appended")
    i+=1
print("All Testing Images Appended!")

## Converting 3d feature array to 2d for training

In [9]:
features[0].shape
features = np.array(features).reshape(len(features), features[0].shape[0]*features[0].shape[1])

In [10]:
testFeatures = np.array(testFeatures).reshape(len(testFeatures), testFeatures[0].shape[0]*testFeatures[0].shape[1])

## TrainTestSplit of features list (Run for BikeVsHorse)

In [11]:
# features, testFeatures, labels, testLabels = train_test_split(features, labels, test_size=0.2)

## Training different classification models using features extracted from pre-trained Alexnet

### LogisticRegression Model

In [12]:
lrmodel = LogisticRegression(solver = 'saga', max_iter = 1000)
lrmodel.fit(features, labels)

In [13]:
y_pred = lrmodel.predict(testFeatures)
acc = accuracy_score(y_pred, testLabels)
print(acc)

### SVM model

In [14]:
clf = LinearSVC(max_iter = 5000)
clf.fit(features,np.array(labels))

In [15]:
y_pred = clf.predict(testFeatures)
acc = accuracy_score(y_pred, testLabels)
print(acc)

### GNB model

In [16]:
gnb = GaussianNB()
gnb.fit(features, labels)

In [17]:
y_pred = gnb.predict(testFeatures)
acc = accuracy_score(y_pred, testLabels)
print(acc)